In [19]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import csv
import os
import sys
import numpy as np
import collections
import scipy.io
from time import time

# CNN bits
import theano
import theano.tensor as T
import lasagne

# hyperopt bits
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL

# for evaluation
sys.path.append(os.path.expanduser('~/projects/engaged_hackathon/'))
from engaged.features import evaluation
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

import lasagne_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Data input

In [21]:
datapath = '/home/michael/projects/engaged_hackathon_data/detection/train_test_patches/'

X_train, y_train, X_val, y_val, X_test, y_test = \
    lasagne_helpers.load_dataset(datapath + 'small.mat')

In [22]:
print X_train.shape
print X_val.shape
print X_test.shape

(2800, 1, 75, 19)
(1200, 1, 75, 19)
(2000, 1, 75, 19)


In [23]:
from math import log
space = (
    hp.uniform( 'drop_input', 0.0, 0.5),
    hp.quniform( 'conv_depth', 1, 1, 1),
    hp.quniform('num_filters1', 32, 128, 1 ),
    hp.quniform('num_filters2', 32, 128, 1 ),
    hp.quniform( 'pool_size', 2, 5, 1 ),
    hp.quniform( 'dense_depth', 1, 4, 1),
    hp.quniform( 'dense_width', 500, 1000, 1),
    hp.uniform( 'drop_dense_hidden', 0.2, 0.8), 
    hp.quniform('log2_minibatch_size', 4, 10, 1 ),
    hp.uniform( 'learning_rate',  0.01 ,  0.1 ), #0.01 - 0.1
    hp.uniform( 'momentum', 0.9, 0.999), #0.9 - 1
#     hp.loguniform( 'learning_rate', log( 0.01 ), log( 0.1 )), #0.01 - 0.1
#     hp.loguniform( 'momentum', log(0.9), log(0.999)), #0.9 - 1
)

In [24]:
print 2**np.array([4, 5,6 ,7, 8, 9, 10])

[  16   32   64  128  256  512 1024]


In [25]:
num_epochs = 15

def run_test( params ):

    """
    
    """
    # extract parameters
    drop_input, conv_depth, num_filters1, num_filters2, pool_size, dense_depth, \
        dense_width, drop_dense_hidden, log2_minibatch_size, learning_rate, momentum = params
    
    minibatch_size = int(2**log2_minibatch_size)
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')

        # create the network
    network = lasagne_helpers.build_cnn(input_var,
        im_width=X_train.shape[2],
        im_height=X_train.shape[3],
        drop_input=drop_input,
        conv_depth=conv_depth,
        num_filters1=num_filters1, 
        num_filters2=num_filters2, 
        pool_size=pool_size, 
        dense_depth=dense_depth,
        dense_width=dense_width, 
        drop_dense_hidden=drop_dense_hidden)

    
    # 
    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()
    # We could add some weight decay as well here, see lasagne.regularization.

    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=learning_rate, momentum=learning_rate)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout layers.
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    # As a bonus, also create an expression for the classification accuracy:
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    # Compile a second function computing the validation loss and accuracy:
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    predict_fn = theano.function([input_var], T.argmax(test_prediction, axis=1))
    
    # Finally, launch the training loop.
    
#     print "  epoch    train loss    valid loss    train/val    valid acc  dur"
    
    # We iterate over epochs:
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time()
        for count, batch in enumerate(lasagne_helpers.iterate_balanced_minibatches(X_train, y_train, int(minibatch_size))):
            print "Batch %d " % count,
            sys.stdout.flush()
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_acc = 0
        val_batches = 0
        y_preds, y_gts = [], []
        for batch in lasagne_helpers.iterate_balanced_minibatches(
                X_val, y_val, int(minibatch_size)):
            
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1
            
            y_preds.append( predict_fn(inputs))
            y_gts.append(targets)
            
        norm_acc = lasagne_helpers.normalised_accuracy(
            np.hstack(y_gts), np.hstack(y_preds))

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))
        print("  normalised val acc:\t\t{:.2f} %".format(
            norm_acc))
        sys.stdout.flush()
              
    return norm_acc

    

    

In [27]:
# headers = [ 'test_accuracy', 'hidden_units', 'learning_rate', 'momentum', 'patience', 'filter1', 'filter2', 'batch_size', 'drop_out', 'epochs', 'conv_1', 'conv_2']

run_counter = 0
def run_wrapper( params ):
    """
    This is the wrappper which writes params to file, and which is minimised"""
    global run_counter
    global o_f

    run_counter += 1
    print "run", run_counter
    sys.stdout.flush()
    #print "params: {}s\n".format(str(params))

    s = time()
#     normalised_accuracy = run_test( params )
    try:
        normalised_accuracy = run_test( params )
        print "normalised_accuracy:", normalised_accuracy
        print "elapsed: {}s \n".format( int( round( time() - s )))
        writer.writerow( [ normalised_accuracy ] + list( params ))
        o_f.flush()
        return {'loss': -normalised_accuracy,
               'status': STATUS_OK}
    except Exception as e:
        print "FAILURE", str(e)
        print "elapsed: {}s \n".format( int( round( time() - s )))
        writer.writerow( [ 'FAILURE' ] + list( params ))
        o_f.flush()
        return {'loss': np.inf,
                'status': STATUS_FAIL,
                'exception': str(e)
               }

output_file = 'hyperopt_log.csv'
max_evals = 250

headers = [ 'norm_acc', 'drop_input', 'conv_depth',
           'num_filters1', 'num_filters2', 'pool_size', 'dense_depth', 
        'dense_width', 'drop_dense_hidden', 'log2_minibatch_size', 'learning_rate', 
           'momentum', 'hidden_units2', 'learning_rate', 'momentum', 'epochs']

o_f = open( output_file, 'wb' )
writer = csv.writer( o_f )
writer.writerow( headers )
o_f.flush()

start_time = time()
best = fmin( run_wrapper, space, algo = tpe.suggest, max_evals = max_evals )
end_time = time()

print "Seconds passed:", int( round( end_time - start_time ))
print "Best run:"
# , optimizer.get_best_run()
print best
print run_test( hyperopt.space_eval( space, best ))


 run 1
0.490382099192 1.0 4.0
75 19
Batch 0  Batch 1  Batch 2  Batch 3  Batch 4  (1024,) (1024,)
Epoch 1 of 15 took 82.337s
  training loss:		0.702518
  validation loss:		0.685881
  validation accuracy:		50.00 %
  normalised val acc:		0.50 %
Batch 0  Batch 1  Batch 2  Batch 3  Batch 4  (1024,) (1024,)
Epoch 2 of 15 took 82.301s
  training loss:		0.690483
  validation loss:		0.677601
  validation accuracy:		56.35 %
  normalised val acc:		0.56 %
Batch 0  Batch 1  Batch 2  Batch 3  Batch 4  (1024,) (1024,)
Epoch 3 of 15 took 82.451s
  training loss:		0.675025
  validation loss:		0.672629
  validation accuracy:		67.19 %
  normalised val acc:		0.67 %
Batch 0  Batch 1  Batch 2  Batch 3  Batch 4  (1024,) (1024,)
Epoch 4 of 15 took 81.479s
  training loss:		0.671130
  validation loss:		0.667622
  validation accuracy:		66.80 %
  normalised val acc:		0.67 %
Batch 0  Batch 1  Batch 2  Batch 3  Batch 4  (1024,) (1024,)
Epoch 5 of 15 took 81.475s
  training loss:		0.662319
  validation loss:		0.664

AssertionError: 